### 1. Data Review & Cleaning

#### 1.1. Initial review - remove redundant columns, standardize column names

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Read dataset
data = pd.read_csv("fifa21_male2.csv")

In [ ]:
# Have a quick look at the data:
data.head()

We notice immediately that the columns `Player Photo`, `Club Logo`, and `Flag Photo` contain *sofifa* links, which will not help in the current analysis, so we can drop them.

In [ ]:
data.drop(['Player Photo', 'Club Logo', 'Flag Photo'], axis=1, inplace=True)

We also notice there is an ID column which we can use as our index column, after ensuring it doesn't have any duplicate values:

In [ ]:
print(len(data['ID'].unique())/data.shape[0]) # equal to 1, so no duplicates

In [ ]:
data.set_index('ID', inplace=True, drop=True)

We also notice there are many columns names which are abbreviations - so we will rename those to the full name for clarity. To do so, we are using a `.csv` file with the abbreviations and their corresponding meaning, as seen on the *sofifa* website. Alongside, we will also standardize the column names:

In [ ]:
# Change capitalization to lowercase and replace spaces with underscores:
data.columns = data.columns.str.lower()
data.columns = data.columns.str.replace(" ", "_")

In [ ]:
# Read positions.csv into dataframe
positions = pd.read_csv('positions.csv', header=None, index_col=0)

# Change dataframe to series and then dictionary so it can be used to rename columns:
positions = positions.squeeze().to_dict()

# Change column names:
for column in data:
    if column in positions:
        data.rename(columns=positions, inplace=True)

Now, we can quickly check which columns have more than 75% null values, so we can discard them from our analysis:

In [ ]:
def check_null_values(df, threshold=75):
    nulls_percentage = {}
    for column in df:
        number_of_nulls = df[column].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[0], 1)
        if null_percentage >= threshold:
            nulls_percentage[column] = null_percentage
    return nulls_percentage

check_null_values(data)

In [ ]:
# As `loan_date_end` has mostly `NaN` values, we can discard it:
data.drop('loan_date_end', axis=1, inplace=True)

To get a sense of the amount of null values in our data, we can check the maximum percentage of `NaN` values in the dataframe:

In [ ]:
def max_nulls(df):
    nulls_percentage = []
    for column in df:
        number_of_nulls = df[column].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[0], 1)
        nulls_percentage.append(null_percentage)
    return max(nulls_percentage)

max_nulls(data)

Given the maximum amount of nulls is low (2.5%), we can continue with our initial review and get back to replacing null values later. 

Next, we'll review the number of unique values per column and ensure there are no columns with one value only, as they will not add any value to the analysis:

In [ ]:
def check_unique_values(df):
    single_value_columns = []
    for column in df:
        if len(df[column].unique()) == 1:
            single_value_columns.append(column)
    return single_value_columns

check_unique_values(data)

In [ ]:
# Check the unique values in the gender column
data['gender'].unique()

In [ ]:
# Remove the gender column as the data shows only male players
data.drop('gender', axis=1, inplace=True)

After looking once more at the data, we observe that the `team_&_contract` column seems to have similar information to the `club` & `contract` columns, so we might be able to remove it too. However, we first have to check our assumption is correct.

In [ ]:
# Create a joined column to compare to the team & contract column
for i in range(0, data.shape[0]):
    if data['contract'].iloc[i] == np.nan:
        data['contract'].iloc[i] = ' '
    if data['club'].iloc[i] == np.nan:
        data['club'].iloc[i] = ' '

data['club_&_contract'] = data['club'] + ' ' + data['contract']

In [ ]:
# Check that the columns are identical and save non-identical values to a dataframe
def check_identical_columns(col1, col2, df):
    diff_values = pd.DataFrame({col1:[], col2:[]})
    identical = 1

    for i in range(0, df.shape[0]):
        if df[col1].iloc[i] == df[col2].iloc[i]:
            continue
        else:
            diff_values.loc[len(diff_values.index)] = [df[col1].iloc[i], df[col2].iloc[i]]
            identical = 0
            

    if identical == 0:
        return diff_values
    else:
        return 'Columns are identical.'         
            
diff_values = check_identical_columns(col1='club_&_contract', col2='team_&_contract', df=data)        

diff_values

We notice two things:
- the `club_&_contract` column has null values, whereas the `team_&_contract` column doesn't
* the `club_&_contract` rows that are non-identical also contain the country that the club belongs to

As the number of values containing the country are low compared to the number of rows in the dataframe, we'll quickly check if the null values come from the `club` and / or `contract` column to see if we can discard the `team_&_contract` column:

In [ ]:
# Check the rows with null values
diff_values[diff_values['club_&_contract'].isna()]

# Check if the nulls come from the contract or club columns:
print(data['contract'].isna().sum())
print(data['club'].isna().sum())

It seems that the null values in the `club_&_contract` column came from the `club` column. However, given that the `team_&_contract` values corresponding to those rows also contain only the contract information, we can remove it, as it doesn't provide any additional information.

In [ ]:
data.drop(['team_&_contract', 'club_&_contract'], axis=1, inplace=True)

#### 1.2. Data Cleaning

##### 1.2.1. Numerical Data Cleaning

`value`, `wage`, and `release clause` columns:

In [ ]:
financials = ['value', 'wage', 'release_clause']

def clean_value(i):
    x = float(i.replace(".","").replace("€","").replace("K","000").replace("M","00000"))
    return x

for column in financials:
    data[column] = data[column].apply(clean_value)

`weight` column:

In [ ]:
def clean_weight(i):
    x = float(i.replace('lbs',''))
    return x

data["weight"] = data["weight"].apply(clean_weight)

`height` column:

In [ ]:
def convert_height(i):
    to_cm = 2.54
    x = i.replace("'"," ")
    x2 = x.replace('"','') # need to do it in two parts 
                           # because of different quote used 
                           # for inch and foot 
    y = x2.split()
    height = round(((float(y[0])*12)+float(y[1]))*to_cm,0)
    return height

data['height'] = data['height'].apply(convert_height)
data['height']

Cleaning the positions columns, i.e. `left-striker`, `goalkeeper`, etc. :

In [ ]:
def cleaning_positions(i):
    x = float(i.replace("+",".").replace("-",""))
    return x

for col in data.loc[:, 'left_striker':'goalkeeper']:
    data[col] = data[col].apply(cleaning_positions)
data.head()

`weak_foot`, `skill_moves`, and `international_reputation` columns:

In [ ]:
star_columns = ['weak_foot', 'skill_moves', 'international_reputation']

# Check unique values
for column in star_columns:
    print(data[column].unique())

In [ ]:
# Select only the first character from the string, then convert the data type to integer
# and check the operation was successful
for column in star_columns:
    data[column] = data[column].str[0]
    data[column] = pd.to_numeric(data[column], errors='raise')
    print(data[column].dtypes)

##### 1.2.2. Categorical Data Cleaning

Extracting the contract ending data from the `contract` column:

In [ ]:
# Explore the contract data
data['contract'].unique()

# As the end of the contract is typically represented by the last 4 characters of the 
# strings, we will extract those where possible:
def clean_contract(x):
    try:
        x = int(x[-4:])
    except:
        pass
    return x

data['contract'] = data['contract'].apply(clean_contract)

# Check what non-integer values remained in the column:
data['contract'].unique()

The values having `On Loan` at the end mean that the player still has a contract with the club, but he's borrowed to a different team in the meantime.

In [ ]:
# Check similarities between columns
print(data.loc[:, ['contract', 'joined', 'nationality']].head(50))
print(data.loc[:, ['contract', 'joined', 'nationality']].tail(50))

In [ ]:
# We'll now remove the 'On Loan' string from the contract column to extract the year and
# replace all 'Country Free' values with NaN:
def clean_loans(x):
    try:
        x = int(x.replace(" On Loan", "")[-4:])
    except:
        x = np.nan
    return x    

data['contract'] = data['contract'].apply(clean_loans)

# Check the operation was successful
data['contract'].unique()

`nationality` column:

In [ ]:
data["nationality"] = data["nationality"].apply(lambda x: "Democratic Republic of the Congo" if str(x).startswith("DR")
                                                 else "North Korea" if str(x).endswith("DPR")
                                                 else "China" if str(x).endswith("PR")
                                                 else str(x).replace("&amp;","and") if "&amp;" in x
                                                 else x)
# data['nationality'].unique()